# Sentiment Analysis for Reddit users on Nuclar Energy

When Enrico Fermi built the first nuclear reactor at the University of Chicago, it was still an elusive technology that promissed a lot. After the World-War II, and the dropping og the nuclear bombs over Hiroshima and Nagasaki, the world was shocked at the destructive power of atomic energy. This hoever, did not prevent governments to engage in masive Nuclear power-plant project in the hope of electrifying their economies. 

In [ ]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from nltk.sentiment import SentimentIntensityAnalyzer
import requests
import time

import praw
from datetime import datetime, timedelta
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

It took me a while to set up the reddit app. It was also not properly working at the beginnig. 

In [ ]:
reddit = praw.Reddit(
    client_id="NQryrzEJMx9V_p7_7nTO-g",
    client_secret="prCOqfgHb8QaRIML3IQzmJYXDx89YA",
    user_agent="EUgoNuclear/1.0 (by /u/Cap12345678)",
    username="u/Cap12345678"
)

In [ ]:
subreddits = ['NuclearEnergy', 'nuclear', 'NuclearPower', 'environment']
posts = []

for subreddit_name in subreddits:
    subreddit = reddit.subreddit(subreddit_name)
    for submission in subreddit.top(limit=100000):
        posts.append([submission.title, submission.selftext, submission.score, submission.created_utc])

df = pd.DataFrame(posts, columns=['title', 'content', 'score', 'timestamp'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s') 
df.to_csv('reddit_nuclear_energy.csv')

In [ ]:
df

In [ ]:
from textblob import TextBlob
import matplotlib.pyplot as plt

def classify_sentiment(text):
    sentiment_score = TextBlob(str(text)).sentiment.polarity
    if sentiment_score > 0:
        return 'positive'
    elif sentiment_score < 0:
        return 'negative'
    else:
        return 'neutral'

df['sentiment_category'] = df['content'].apply(classify_sentiment)

In [ ]:
positive_trend = df[df['sentiment_category'] == 'positive'].resample('M', on='timestamp').size()
neutral_trend = df[df['sentiment_category'] == 'neutral'].resample('M', on='timestamp').size()
negative_trend = df[df['sentiment_category'] == 'negative'].resample('M', on='timestamp').size()

In [ ]:
plt.plot(positive_trend.index, positive_trend.values, label='Positive')
#plt.plot(neutral_trend.index, neutral_trend.values, label='Neutral')
plt.plot(negative_trend.index, negative_trend.values, label='Negative')
plt.xlabel('Date')
plt.ylabel('Count')
plt.title('Sentiment Trend over Time')
plt.legend()
plt.show()

In [ ]:
#Bag of Words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

X = df['title']
y = df['score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_test)

In [ ]:
#Step 5: Training Decision Tree Regressor

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

hyperparameters = [{'max_depth': 3}, {'max_depth': 5}, {'max_depth': 10}]

for params in hyperparameters:
    regressor = DecisionTreeRegressor(**params)
    regressor.fit(X_train_counts, y_train)
    
    y_pred_train = regressor.predict(X_train_counts)
    y_pred_test = regressor.predict(X_test_counts)
    
    print(f"Hyperparameters: {params}")
    print(f"Training MSE: {mean_squared_error(y_train, y_pred_train)}")
    print(f"Test MSE: {mean_squared_error(y_test, y_pred_test)}\n")


In [ ]:
'''PF1:chapter 3 slides'''

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import datetime as dt
import pandas_datareader.data as web 
import os
import yfinance as yf
yf.pdr_override()
plt.rc('figure', figsize=(10, 6)) 
pd.options.display.max_rows = 20 
np.set_printoptions(precision=4, suppress=True)

In [ ]:
import yfinance as yf
import pandas as pd

# List of ticker symbols 
'''NextEra Energy, Inc. (NEE), Cameco Corporation, Energy Fuels Inc., Uranium Energy Corp., 
Public Service Enterprise Group Incorporated (PEG), Entergy Corporation (ETR), and Denison Mines Corp.'''
tickers = ['NEE', 'CCJ', 'UUUU', 'UEC', 'PEG', 'ETR', 'DNN']

# Dates
start_date = '2013-01-01'
end_date = '2023-08-11'

# Create an empty dictionary to store the dataframe
stock_data = {}

In [ ]:
for ticker in tickers:
    # Download data
    data = yf.download(ticker, start=start_date, end=end_date)
    
    # Store in dictionary
    stock_data[ticker] = data


In [ ]:
# Print the number of non-missing observations
for ticker, data in stock_data.items():
    print(f"{ticker}: {data['Close'].count()} non-missing observations")

In [ ]:
print(stock_data)

In [ ]:
for stock in stock_data:
    print(stock, stock_data[stock].count())

In [ ]:
# Create an empty DataFrame to store the rescaled adjusted closing prices
rescaled_data = pd.DataFrame()

for ticker, data in stock_data.items():
    # Find the price at the end of 2021
    end_of_2021_price = data['Adj Close']['2021-12-31']
    
    # Rescale the adjusted closing prices
    rescaled_adjusted_close = data['Adj Close'] / end_of_2021_price
    
    # Add to the rescaled_data DataFrame
    rescaled_data[ticker] = rescaled_adjusted_close

# Filter the data for the period from 2022 to 2023
rescaled_data_2022_2023 = rescaled_data['2022-01-01':'2023-08-11']


In [ ]:
# Plot the rescaled adjusted closing prices
rescaled_data_2022_2023.plot(figsize=(15,10), title="Rescaled Adjusted Closing Prices (2022-2023)")
plt.xlabel("Date")
plt.ylabel("Rescaled Price")
plt.legend(title="Stocks")
plt.show()


In [ ]:
# Create a new DataFrame to store the calculated values
calculated_data = pd.DataFrame()

for ticker, data in stock_data.items():
    # Calculate the daily arithmetic returns
    data['Daily Return'] = data['Adj Close'].pct_change()

    # Calculate the intra-day volatility
    data['Intra-Day Volatility'] = (data['High'] - data['Low']) / data['Open']

    # Select only the relevant columns
    calculated_data[ticker + '_Return'] = data['Daily Return']
    calculated_data[ticker + '_Volatility'] = data['Intra-Day Volatility']


In [ ]:
# Loop through the tickers and create scatter plots
for ticker in tickers:
    plt.figure(figsize=(10,6))
    plt.scatter(calculated_data[ticker + '_Volatility'], calculated_data[ticker + '_Return'], alpha=0.5)
    plt.title(f'Scatter Plot for {ticker}')
    plt.xlabel('Intra-Day Volatility')
    plt.ylabel('Daily Return')
    plt.grid(True)
    plt.show()


In [ ]:
# Calculate the equal-weighted portfolio value
portfolio_value = rescaled_data.sum(axis=1) / len(tickers)

# Filter the data for the period from 2021 to 2023
portfolio_value_2022_2023 = portfolio_value['2022-01-01':'2023-08-11']

In [ ]:
plt.figure(figsize=(15,10))
portfolio_value_2022_2023.plot(title="Equal-Weighted Portfolio Value (2022-2023)")
plt.xlabel("Date")
plt.ylabel("Portfolio Value")
plt.grid(True)
plt.show()

Analyzing stock performance over a long period of time can provide valuable insights, especially if one is looking for any discontinuities or abnormal behavior. A discontinuity could be the result of significant events like mergers, acquisitions, corporate restructuring, financial crises, or major market news that affect the stock price.

Here, I plot the equal-weighted portfolio value from 2013 to 2023 with an emphasis on February 2022 to visually inspect any discontinuity.

Equal-weighted portfolio value from 2013 to 2023, with a vertical dashed line at the end of February 2022 is shown below. Significant changes in sentiment can be noticed as of early 2020, but this could mostly be attributed to the COVID shutdown, and major energy portfolio readjustments due to lockdown and grounding of airlines. A second change, however is noticed just prior to Russias invasion of Ukraine, and this could be interpreted as a resilience in sentiment, despite Russias repeated  to nuclear threats, and its assaults in and near Zaporizhzhia Nuclear Power Plant.

Visual inspection provides an initial understanding but might not be conclusive. So I might need to apply statistical tests (RDD, for example).

In [ ]:
plt.figure(figsize=(15,10))
portfolio_value.plot(title="Equal-Weighted Portfolio Value (2013-2023)")
plt.axvline(x='2022-02-28', color='red', linestyle='--', label='February 2022 Cutoff')
plt.xlabel("Date")
plt.ylabel("Portfolio Value")
plt.legend()
plt.grid(True)
plt.show()

# Regression Discontinuity Design

This is not to test my understanding on RDD, but rrather an atempt to run this model on Python, given the data at hand. My thinking was if I can run the model on scraped data, but I could not come up with examples to follow. I use a paper




\begin{thebibliography}{99}

\bibitem{smith2020}
Smith, J., Johnson, K., \& Williams, L. (2020). Title of the paper. Journal Name, Volume(Issue), Pages.

\end{thebibliography}

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
import pandas as pd

In [ ]:
# Assume you have a DataFrame 'df' with columns 'outcome', 'treatment', 'running_var'
# 'outcome' is the adjusted price (portfolio value)
# 'treatment' is the binary treatment indicator (0 or 1 for war in Ukraie)
# 'running_var' is the running variable used to define the threshold

# Treatment start date
treatment_start_date = '2022-02-24'

# Create DataFrame
rdd_data = pd.DataFrame({
    'outcome': portfolio_value, # Portfolio performance
    'running_var': portfolio_value.index.to_julian_date(),
})

# Assign the treatment based on the treatment start date
rdd_data['treatment'] = (rdd_data.index >= treatment_start_date).astype(int)

# Filter the DataFrame to the relevant period
rdd_data = rdd_data['2013-01-01':'2023-08-11']

# Print the first few rows to check
print(rdd_data.head())
print(rdd_data.tail())

In [ ]:

import seaborn as sns

# Fit the RDD model

In [ ]:
# Fit the RDD model
# Define the threshold value for the running variable
threshold_value = pd.Timestamp(treatment_start_date).to_julian_date()

# Create an indicator for observations to the right of the threshold
rdd_data['threshold_indicator'] = (rdd_data['running_var'] > threshold_value).astype(int)

# Interact the treatment indicator with the threshold indicator
rdd_data['interaction'] = rdd_data['treatment'] * rdd_data['threshold_indicator']

# Define the regression model including the interaction term
model_formula = "outcome ~ treatment + threshold_indicator + interaction + running_var"

# Fit the regression model using Ordinary Least Squares (OLS)
result = smf.ols(formula=model_formula, data=rdd_data).fit()


In [ ]:
# Create a DataFrame with the running variable and predicted outcome
plot_data = pd.DataFrame({
    'running_var': rdd_data['running_var'],
    'predicted_outcome': result.predict(rdd_data),
})

In [ ]:
# Plot the actual outcome
plt.scatter(rdd_data['running_var'], rdd_data['outcome'], color='blue', alpha=0.5, label='Actual Outcome')
# Plot the predicted outcome (fitted lines)
sns.lineplot(x='running_var', y='predicted_outcome', data=plot_data, color='red', label='Fitted Line')

# Draw the threshold line
plt.axvline(x=pd.Timestamp(treatment_start_date).to_julian_date(), color='green', linestyle='--', label='Threshold')

# Labels and legend
plt.title('Regression Discontinuity Plot')
plt.xlabel('Running Variable')
plt.ylabel('Outcome (Portfolio Performance)')
plt.legend()
plt.show()'''

In [ ]:
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import IV2SLS



# Define the treatment based on the threshold
rdd_data['treatment'] = (rdd_data['running_var'] >= threshold_value).astype(int)

# Add a constant term
rdd_data['const'] = 1

# Define endogenous variable (outcome)
endog = rdd_data['outcome']

# Define exogenous variables
exog = rdd_data[['const', 'running_var', 'treatment']]

# Define the instrument (just the running variable in this simplified model)
instrument = rdd_data[['const', 'running_var']]

# Perform the 2SLS regression
model_2sls = IV2SLS(endog, exog, instrument=instrument).fit()

In [ ]:
# Print the summary
print(model_2sls.summary())

In [ ]:
# Predicted outcome based on the 2SLS fit
rdd_data['predicted_outcome'] = model_2sls.predict(exog)

# Separate data above and below the threshold
below_threshold = rdd_data[rdd_data['running_var'] < threshold_value]
above_threshold = rdd_data[rdd_data['running_var'] >= threshold_value]

# Plot actual outcomes
plt.scatter(rdd_data['running_var'], rdd_data['outcome'], color='blue', alpha=0.5, label='Actual Outcome')

# Plot fitted lines for below and above the threshold
plt.plot(below_threshold['running_var'], below_threshold['predicted_outcome'], color='red', label='Fitted Line (Below Threshold)')
plt.plot(above_threshold['running_var'], above_threshold['predicted_outcome'], color='green', label='Fitted Line (Above Threshold)')

# Draw the threshold line
plt.axvline(x=threshold_value, color='black', linestyle='--', label='Threshold')

# Labels and legend
plt.title('Fuzzy Regression Discontinuity Plot')
plt.xlabel('Running Variable')
plt.ylabel('Outcome (Portfolio Performance)')
plt.legend()
plt.show()
